# **Historical Waste Dataset Generation**

The historical dataset was generated using report-based baseline indicators derived from Rwanda’s waste management literature. Population data for all Kigali sectors were obtained from the 2022 Rwanda Population and Housing Census published by the National Institute of Statistics of Rwanda (NISR).

Daily baseline waste generation was computed using the Kigali per-capita waste generation rate reported by the Global Green Growth Institute (GGGI, 2023), estimated at 0.56 kg per capita per day. The baseline daily waste for each zone was calculated using:


\[
\text{Waste (tons/day)} = \frac{\text{Population} \times 0.56}{1000}
\]




A continuous daily time series was generated for the period 2021–2023. Contextual temporal features were included to support forecasting, including day-of-week, month, and an Umuganda indicator representing the last Saturday of each month, which reflects localized variations in waste generation patterns.

The resulting dataset forms the baseline historical input for the forecasting component of the WI-DR system, providing structured daily waste estimates for each sector prior to preprocessing, feature engineering, and model training.


The resulting dataset forms the baseline historical input for the forecasting component of the WI-DR system, providing structured daily waste estimates for each pilot zone prior to preprocessing, feature engineering, and model training.


In [ ]:
#Import neccessary libaries
import pandas as pd
from datetime import timedelta
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path to your population CSV
POP_PATH = "/content/drive/MyDrive/WI-DR/kigali_sectors_population.csv"

# Dataset time range
START_DATE = "2021-01-01"
END_DATE   = "2023-12-31"

# Kigali waste generation rate (kg per capita per day)
KG_PER_CAPITA_PER_DAY = 0.56


# LOAD POPULATION DATA
pop_df = pd.read_csv(POP_PATH)
pop_df.columns = [c.strip().lower() for c in pop_df.columns]

print("Detected columns:", pop_df.columns)

# Use correct columns and rename
zones = pop_df[["sector_id", "sector_name", "population_2022"]].copy()
zones.rename(columns={
    "sector_id": "zone_id",
    "sector_name": "zone_name",
    "population_2022": "population"
}, inplace=True)

# Clean population
zones["population"] = pd.to_numeric(zones["population"], errors="coerce")
zones = zones.dropna(subset=["population"])
zones["population"] = zones["population"].astype(int)

# Compute baseline daily waste (tons)
zones["waste_baseline_tons"] = (zones["population"] * KG_PER_CAPITA_PER_DAY) / 1000.0


# GENERATE DATE RANGE
dates = pd.date_range(START_DATE, END_DATE, freq="D")


# UMUGANDA DETECTION
def is_last_saturday(d):
    if d.weekday() != 5:
        return 0
    return 1 if (d + timedelta(days=7)).month != d.month else 0


# BUILD DATASET
rows = []

for d in dates:
    dow = d.weekday()
    month = d.month
    umug = is_last_saturday(d)

    for _, r in zones.iterrows():
        rows.append({
            "date": d.strftime("%Y-%m-%d"),
            "zone_id": r["zone_id"],        # SEC_01 etc
            "zone_name": r["zone_name"],    # REAL SECTOR NAME
            "population": int(r["population"]),
            "waste_baseline_tons": round(float(r["waste_baseline_tons"]), 6),
            "dow": dow,
            "month": month,
            "is_umuganda": umug
        })

hist = pd.DataFrame(rows)


# SAVE DATASET
OUTPUT_FILE = "historical_waste_kigali_sectors.csv"
hist.to_csv(OUTPUT_FILE, index=False)


# OUTPUT CHECK
print("Zones:", zones.shape[0])
print("Dataset shape:", hist.shape)
print(hist.head())
print("\nSaved to:", OUTPUT_FILE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Detected columns: Index(['sector_id', 'sector_name', 'population_2022', 'district',
       'pop_share_2022', 'sector_msw_tpd_baseline'],
      dtype='object')
Zones: 35
Dataset shape: (38325, 8)
         date zone_id   zone_name  population  waste_baseline_tons  dow  \
0  2021-01-01  SEC_01      Gitega       26668             14.93408    4   
1  2021-01-01  SEC_02    Kanyinya       31026             17.37456    4   
2  2021-01-01  SEC_03      Kigali       61499             34.43944    4   
3  2021-01-01  SEC_04  Kimisagara       56534             31.65904    4   
4  2021-01-01  SEC_05  Mageregere       59747             33.45832    4   

   month  is_umuganda  
0      1            0  
1      1            0  
2      1            0  
3      1            0  
4      1            0  

Saved to: historical_waste_kigali_sectors.csv


# **Contextual Data Integration — Weather Dataset**

To improve waste-demand prediction accuracy, daily weather data was integrated as an exogenous input into the WI-DR forecasting pipeline. Weather conditions influence waste generation through moisture effects, decomposition rates, and operational disruptions. The weather dataset was generated using the Open-Meteo Historical Archive API for Kigali, Rwanda.



### Location Parameters
Weather data was retrieved using the geographic coordinates of Kigali:

- Latitude: -1.9441  
- Longitude: 30.0619  

These coordinates represent central Kigali and provide city-level daily weather signals suitable for zone-day modeling.



### Data Source
Open-Meteo Historical Weather Archive

Endpoint used:
https://archive-api.open-meteo.com/v1/archive

---

### Weather Variables Collected

| Variable (API) | Dataset Column | Description |
|---------------|---------------|-------------|
| precipitation_sum | rain_mm | Total daily rainfall (mm) |
| temperature_2m_mean | temp_c | Mean daily temperature (°C) |
| relative_humidity_2m_mean | humidity | Mean daily relative humidity (%) |

In [ ]:
import requests

# Define location (Kigali coordinates)
LAT = -1.9441   # Latitude of Kigali, Rwanda
LON = 30.0619   # Longitude of Kigali, Rwanda

# Define time range for data extraction
START = "2021-01-01"   # Start date of historical weather data
END = "2023-12-31"     # End date of historical weather data

# Build Open-Meteo API request URL
# This request asks for daily weather data including:
# Mean temperature, total daily precipitation, and mean relative humidity.
# Timezone is set to auto so dates align with Kigali local time.
url = (
    "https://archive-api.open-meteo.com/v1/archive"
    f"?latitude={LAT}"
    f"&longitude={LON}"
    f"&start_date={START}"
    f"&end_date={END}"
    "&daily=temperature_2m_mean,precipitation_sum,relative_humidity_2m_mean"
    "&timezone=auto"
)

# Send request to Open-Meteo API
response = requests.get(url)     # Fetch weather data from API
data = response.json()           # Convert JSON response into Python dictionary

# Convert JSON to structured DataFrame
df = pd.DataFrame({
    "date": data["daily"]["time"],                          # Observation date
    "rain_mm": data["daily"]["precipitation_sum"],          # Daily rainfall (mm)
    "temp_c": data["daily"]["temperature_2m_mean"],         # Mean temperature (°C)
    "humidity": data["daily"]["relative_humidity_2m_mean"]  # Mean relative humidity (%)
})

# Export dataset to CSV for integration into WI-DR pipeline
df.to_csv("weather_daily_kigali.csv", index=False)

# Confirmation message
print("Weather dataset with humidity generated.")


Weather dataset with humidity generated.


# **Contextual Data Integration — Calendar & Holiday Events Dataset**

To improve waste-demand prediction accuracy, calendar-based event signals were integrated as an exogenous input into the WI-DR forecasting pipeline. Public holidays and community activity days influence waste generation through changes in household behavior, commercial activity, and temporary waste spikes. The holiday dataset was generated by extracting official Rwanda public holidays for the period 2021–2023.



**Coverage Period**

Calendar events were collected for the full modeling horizon: January 1, 2021 → December 31, 2023. Daily resolution


**Data Source**

Rwanda Public Holiday Calendar — TimeandDate

Pages used:

https://www.timeanddate.com/holidays/rwanda/2021

https://www.timeanddate.com/holidays/rwanda/2022

https://www.timeanddate.com/holidays/rwanda/2023

Only official public holidays were retained; observances and non-official events were excluded.

Event Variables Collected:
Raw Column	Dataset Column	Description
date	date	Calendar date (YYYY-MM-DD)
holiday name	name	Official holiday name
source	source	Data origin identifier
Derived Calendar Features (Used in WI-DR)

After integration into the zone-day dataset, the following modeling features were generated:


Feature	Description
is_holiday	Binary indicator (1 if public holiday, else 0),
event_count_cal	Number of calendar events on that day, event_intensity_cal	Weighted influence of calendar events on waste demand

In [ ]:
 import re
import pandas as pd

# URLs for Rwanda holidays by year (Timeanddate)
YEAR_URLS = {
    2021: "https://www.timeanddate.com/holidays/rwanda/2021?hol=1",
    2022: "https://www.timeanddate.com/holidays/rwanda/2022?hol=1",
    2023: "https://www.timeanddate.com/holidays/rwanda/2023?hol=1",
}

def normalize_cols(cols):
    """
    Flatten and normalize column names to lowercase strings.
    Handles MultiIndex headers and cleans extra spaces/symbols.
    """
    # Flatten MultiIndex headers
    if isinstance(cols, pd.MultiIndex):
        flat = []
        for tup in cols.values:
            joined = " ".join([str(x) for x in tup if x and str(x) != "nan"]).strip()
            flat.append(joined)
    else:
        flat = [str(c).strip() for c in cols]

    # Lowercase + clean non-alphanumeric (keeps spaces)
    clean = []
    for c in flat:
        c = c.lower()
        c = re.sub(r"[^a-z0-9 ]+", "", c)  # remove punctuation
        c = re.sub(r"\s+", " ", c).strip() # collapse spaces
        clean.append(c)

    return clean

def has_date_and_name_like(cols_norm):
    """
    Returns True if columns contain something like:
    - date (e.g. 'date', 'date day', etc.)
    - and name/holiday (e.g. 'name', 'holiday', 'holiday name')
    """
    has_date = any("date" in c for c in cols_norm)
    has_name = any(("name" in c) or ("holiday" in c) for c in cols_norm)
    return has_date and has_name

all_rows = []

for year, url in YEAR_URLS.items():

    # Pull all tables from the page
    tables = pd.read_html(url)

    chosen = None
    chosen_cols = None

    # Try to find the holiday table using "contains" matching
    for t in tables:
        cols_norm = normalize_cols(t.columns)

        if has_date_and_name_like(cols_norm):
            chosen = t.copy()
            chosen_cols = cols_norm
            break

    # If still not found, print table column options for debugging
    if chosen is None:
        print(f"\n[DEBUG] Could not auto-detect holiday table for {year}.")
        print("These are the tables I found and their normalized columns:\n")
        for i, t in enumerate(tables):
            print(i, normalize_cols(t.columns))
        raise RuntimeError(f"Could not find holiday table for year {year} at {url}")

    # Apply normalized columns
    chosen.columns = chosen_cols

    # Identify which column is the "date-like" column
    date_col = next(c for c in chosen.columns if "date" in c)

    # Identify which column is the "name/holiday-like" column
    name_col = next(c for c in chosen.columns if ("name" in c) or ("holiday" in c))

    # If a "type" column exists, keep only "Public Holiday"

    if "type" in chosen.columns:
        chosen = chosen[
            chosen["type"].astype(str).str.contains("Public Holiday", case=False, na=False)
        ]

    # Parse dates
    def parse_date(x):
        x = str(x).strip()
        if str(year) in x:
            return pd.to_datetime(x, errors="coerce")
        return pd.to_datetime(f"{x} {year}", errors="coerce")

    chosen["date"] = chosen[date_col].apply(parse_date)
    chosen = chosen.dropna(subset=["date"])

    # Build output table
    out = pd.DataFrame({
        "date": chosen["date"],
        "name": chosen[name_col].astype(str).str.strip(),
        "source": "timeanddate",
    })

    all_rows.append(out)

# Combine all years + export
holidays = (
    pd.concat(all_rows, ignore_index=True)
      .drop_duplicates(subset=["date", "name"])
      .sort_values("date")
)

holidays["date"] = holidays["date"].dt.strftime("%Y-%m-%d")
holidays.to_csv("rwanda_holidays.csv", index=False)

print("\n Saved: rwanda_holidays.csv")



 Saved: rwanda_holidays.csv


In [ ]:
import pandas as pd

baseline = pd.read_csv("historical_waste_kigali_sectors.csv")
baseline["date"] = pd.to_datetime(baseline["date"])

hol = pd.read_csv("rwanda_holidays.csv")
hol["date"] = pd.to_datetime(hol["date"])
holiday_set = set(hol["date"].dt.date)

baseline["is_holiday"] = baseline["date"].dt.date.isin(holiday_set).astype(int)

# Umuganda = last Saturday of each month
def is_last_saturday(dt):
    return int(dt.weekday() == 5 and (dt + pd.Timedelta(days=7)).month != dt.month)

baseline["is_umuganda"] = baseline["date"].apply(is_last_saturday)

# Calendar-based event_count and intensity (deterministic, real)
baseline["event_count_cal"] = baseline["is_holiday"] + baseline["is_umuganda"]

# Intensity weights
baseline["event_intensity_cal"] = (
    baseline["is_umuganda"] * 40 +
    baseline["is_holiday"] * 70
).clip(0, 100).astype(float)

out = baseline[["date", "zone_id", "event_count_cal", "event_intensity_cal"]].copy()
out["date"] = out["date"].dt.strftime("%Y-%m-%d")
out.to_csv("events_calendar_zone_daily_2021_2023.csv", index=False)

print("Saved: events_calendar_zone_daily_2021_2023.csv")


Saved: events_calendar_zone_daily_2021_2023.csv


# **Contextual Data Integration — Satellite Vegetation (NDVI) Dataset**

To enhance environmental awareness within the WI-DR forecasting pipeline, satellite-derived vegetation signals were integrated as an exogenous input. Vegetation density and seasonal biomass cycles influence organic waste generation, decomposition rates, and environmental conditions across urban zones. The  NDVI (Normalized Difference Vegetation Index) dataset was generated using Sentinel-2 satellite imagery processed through Google Earth Engine for Kigali sectors over the period 2021–2023.

**Coverage Period**

Satellite vegetation data was collected for the full modeling horizon:

January 1, 2021 → December 31, 2023
Daily resolution (derived from monthly satellite observations)

**Data Source**

Sentinel-2 Surface Reflectance (SR) Harmonized — Google Earth Engine

Vegetation index computed using:

NDVI = (B8 − B4) / (B8 + B4)

**Where:**

B8 = Near-Infrared band

B4 = Red band

**Spatial Preparation**

Kigali administrative sector boundaries were processed using GeoPandas:

- Sector column automatically detected and standardized  
- Dataset filtered to Kigali districts *(Gasabo, Kicukiro, Nyarugenge...)*  
- Cleaned and normalized `zone_id` created  
- Polygons dissolved to produce **35 Kigali sectors**  
- Reprojected to **EPSG:4326** and exported as **GeoJSON** and **Shapefile** for Google Earth Engine use  


**Satellite Variables Collected**

| Raw Variable | Dataset Column | Description |
|-------------|---------------|-------------|
| NDVI | ndvi | Vegetation index representing vegetation density and biomass |
| date | date | Observation date |
| zone_id | zone_id | Kigali sector identifier |




**Temporal Processing**

Satellite NDVI was originally available at monthly resolution and processed as follows:


*   Monthly NDVI exported per sector for 2021–2023
*   All yearly datasets merged into one continuous monthly dataset
*   Duplicate records removed and sorted chronologically
*   Monthly NDVI expanded to daily resolution
*   Forward-fill applied per sector to maintain temporal continuity








In [ ]:
import geopandas as gpd
import pandas as pd
import re

# Load file
INPUT_GEOJSON = "kigali_sectors.geojson.json"
gdf = gpd.read_file(INPUT_GEOJSON)

print("Columns:", list(gdf.columns))
print("CRS:", gdf.crs)
print("Rows:", len(gdf))

# Helpers
def clean_text(s):
    s = s.astype(str)
    s = s.str.strip()
    s = s.str.replace(r"\s+", " ", regex=True)  # collapse whitespace
    s = s.str.replace("\u00a0", " ", regex=False)  # non-breaking spaces
    s = s.str.lower()
    return s

def pick_best_zone_col(df, candidates):
    best = None
    best_uniques = None
    for c in candidates:
        if c in df.columns:
            n = clean_text(df[c]).nunique()
            print(f"[candidate] {c}: unique={n}")
            # Prefer something close to 35 (or at least much smaller than 381)
            if best is None or abs(n - 35) < abs(best_uniques - 35):
                best, best_uniques = c, n
    return best, best_uniques

# Choose a sector column
CANDIDATES = [
    "zone_id", "sector", "Sector", "sector_name", "SECTOR",
    "NAME_3", "NAME", "name", "admin3Name", "ADM3_EN", "ADM3_NAME",
    "ADM3", "adm3", "SNAME", "S_NAME", "sectorName"
]

zone_col, zone_uniques = pick_best_zone_col(gdf, CANDIDATES)
if zone_col is None:
    raise ValueError("Could not find a usable sector column. Paste the printed Columns list.")

print("Chosen zone_col:", zone_col, "| uniques:", zone_uniques)

# Filter to Kigali using admin fields
ADMIN_CANDS = ["district", "DISTRICT", "ADM2_NAME", "NAME_2", "province", "PROVINCE", "ADM1_NAME", "NAME_1", "city", "CITY"]
admin_found = [c for c in ADMIN_CANDS if c in gdf.columns]

kigali_districts = {"gasabo", "kicukiro", "nyarugenge"}

if admin_found:
    print("Admin columns found:", admin_found)

    district_cols = [c for c in admin_found if "dist" in c.lower() or "adm2" in c.lower() or "name_2" in c.lower()]
    filtered = False

    for dc in district_cols:
        vals = set(clean_text(gdf[dc]).unique())
        if vals & kigali_districts:
            gdf = gdf[clean_text(gdf[dc]).isin(kigali_districts)].copy()
            print(f"Filtered to Kigali districts using {dc}. Rows now:", len(gdf))
            filtered = True
            break

    if not filtered:
        for ac in admin_found:
            vals = set(clean_text(gdf[ac]).unique())
            if "kigali" in vals:
                gdf = gdf[clean_text(gdf[ac]).eq("kigali")].copy()
                print(f"Filtered to Kigali using {ac} == 'kigali'. Rows now:", len(gdf))
                break

zone_uniques_after = clean_text(gdf[zone_col]).nunique()
print("Unique zone values after filtering:", zone_uniques_after)

# Build zone_id
gdf["zone_id"] = clean_text(gdf[zone_col])

if zone_uniques_after > 60:
    district_like_cols = [c for c in gdf.columns if any(k in c.lower() for k in ["district", "adm2", "name_2"])]
    if district_like_cols:
        dc = district_like_cols[0]
        gdf["zone_id"] = clean_text(gdf[dc]) + "__" + clean_text(gdf[zone_col])
        print(f"Using composite zone_id = {dc} + {zone_col}")
    else:
        print("No district-like column found to build composite key.")

print("Unique zone_id BEFORE dissolve:", gdf["zone_id"].nunique())

# Dissolve
g35 = gdf.dissolve(by="zone_id", as_index=False)

print("Unique zone_id AFTER dissolve:", g35["zone_id"].nunique())

# Save
g35 = g35[["zone_id", "geometry"]].to_crs("EPSG:4326")
g35.to_file("kigali_35_sectors.geojson", driver="GeoJSON")

print("Saved: kigali_35_sectors.geojson")
print(g35.head(5))


Columns: ['GID_3', 'GID_0', 'COUNTRY', 'GID_1', 'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2', 'NL_NAME_2', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3', 'geometry']
CRS: EPSG:4326
Rows: 422
[candidate] NAME_3: unique=381
Chosen zone_col: NAME_3 | uniques: 381
Admin columns found: ['NAME_2', 'NAME_1']
Filtered to Kigali districts using NAME_2. Rows now: 35
Unique zone values after filtering: 35
Unique zone_id BEFORE dissolve: 35
Unique zone_id AFTER dissolve: 35
Saved: kigali_35_sectors.geojson
    zone_id                                           geometry
0   bumbogo  POLYGON ((30.1401 -1.8636, 30.1399 -1.8322, 30...
1   gahanga  POLYGON ((30.1038 -2.0599, 30.1037 -2.0566, 30...
2   gatenga  POLYGON ((30.0785 -2.0269, 30.0783 -2.0318, 30...
3   gatsata  POLYGON ((30.0384 -1.9419, 30.034 -1.9427, 30....
4  gikomero  POLYGON ((30.229 -1.8977, 30.2188 -1.8964, 30....


In [ ]:
import geopandas as gpd
import os, shutil

#  Read the 35-sectors GeoJSON (the dissolved output)
gdf = gpd.read_file("/content/kigali_35_sectors.geojson")

print("Zones:", gdf["zone_id"].nunique(), " | Rows:", len(gdf))  # should be 35

# Write shapefile folder (clean folder first)
out_dir = "/content/kigali_35_sectors_shp"
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir, exist_ok=True)

shp_path = os.path.join(out_dir, "kigali_35_sectors.shp")
gdf.to_file(shp_path, driver="ESRI Shapefile")

#  Zip the shapefile components
zip_path = shutil.make_archive("/content/kigali_35_sectors", "zip", out_dir)
print("Created ZIP:", zip_path)


Zones: 35  | Rows: 35
Created ZIP: /content/kigali_35_sectors.zip


**Merge NIDVI Dataset:** This stage prepares the satellite vegetation (NDVI) dataset for integration into the WI-DR zone-day forecasting dataset. Monthly NDVI values exported from Google Earth Engine for 2021–2023 are first loaded, cleaned, standardized, and merged into a single continuous monthly dataset covering all 35 Kigali sectors. Duplicate records are removed and data is sorted chronologically.

Since the WI-DR forecasting pipeline operates at daily resolution, the monthly NDVI values are expanded into a complete sector-day time series. A full grid of all sector–date combinations is generated, and each day inherits its corresponding monthly NDVI value through forward-filling within each sector. This preserves temporal continuity while avoiding future data leakage. The result is a daily NDVI dataset aligned with the 35 Kigali sectors, ready to be merged with baseline waste, weather, and calendar-event datasets for forecasting and routing optimization.

In [ ]:
import os
import pandas as pd

# Configuration: paths to monthly NDVI files exported from Google Earth Engine
NDVI_FILES = {
    2021: "/content/drive/MyDrive/satellite_zone_monthly_ndvi_2021.csv",
    2022: "/content/drive/MyDrive/satellite_zone_monthly_ndvi_2022.csv",
    2023: "/content/drive/MyDrive/satellite_zone_monthly_ndvi_2023.csv",
}

# Output files
OUT_MONTHLY = "/content/satellite_zone_monthly_ndvi_2021_2023.csv"
OUT_DAILY   = "/content/satellite_zone_daily_2021_2023.csv"

# Date range for expanding to daily resolution
START_DATE = "2021-01-01"
END_DATE   = "2023-12-31"


def load_ndvi_year(path: str) -> pd.DataFrame:
    """
    Load one yearly NDVI file and standardize column names and data types.
    Ensures required columns exist and formats are correct.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

    df = pd.read_csv(path)

    # Sometimes NDVI is exported as "mean" from Earth Engine
    if "ndvi" not in df.columns and "mean" in df.columns:
        df = df.rename(columns={"mean": "ndvi"})

    # Ensure required columns exist
    required = {"zone_id", "date", "ndvi"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"{path} missing columns: {missing}. Found: {list(df.columns)}")

    # Standardize formats
    df["zone_id"] = df["zone_id"].astype(str).str.strip()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["ndvi"] = pd.to_numeric(df["ndvi"], errors="coerce")

    # Remove rows missing key identifiers
    df = df.dropna(subset=["zone_id", "date"])

    return df[["zone_id", "date", "ndvi"]]


def merge_ndvi_years(files_dict: dict) -> pd.DataFrame:
    """
    Merge all yearly NDVI datasets into a single continuous monthly dataset.
    Removes duplicates and sorts chronologically.
    """
    frames = []
    for year, path in sorted(files_dict.items()):
        df = load_ndvi_year(path)
        df["year_source"] = year
        frames.append(df)

    out = pd.concat(frames, ignore_index=True)

    # Remove duplicate zone_id + date entries if any
    out = out.drop_duplicates(subset=["zone_id", "date"], keep="last")

    # Sort by sector and time
    out = out.sort_values(["zone_id", "date"]).reset_index(drop=True)

    return out


def monthly_to_daily_forwardfill(ndvi_monthly: pd.DataFrame,
                                 start_date: str,
                                 end_date: str) -> pd.DataFrame:
    """
    Convert monthly NDVI into daily NDVI by forward-filling values
    within each zone (no future information leakage).
    """
    df = ndvi_monthly.copy()

    # Convert each record to its corresponding month
    df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

    zones = df["zone_id"].unique()
    daily_dates = pd.date_range(start_date, end_date, freq="D")

    # Create full Zone × Day grid
    grid = pd.MultiIndex.from_product(
        [zones, daily_dates],
        names=["zone_id", "date"]
    ).to_frame(index=False)

    grid["month"] = grid["date"].dt.to_period("M").dt.to_timestamp()

    # Merge monthly NDVI into daily grid
    daily = grid.merge(df[["zone_id", "month", "ndvi"]],
                       on=["zone_id", "month"],
                       how="left")

    daily = daily.sort_values(["zone_id", "date"])

    # Forward-fill NDVI per zone (no future-to-past filling)
    daily["ndvi"] = daily.groupby("zone_id")["ndvi"].ffill()

    # Clean output
    daily = daily.drop(columns=["month"])
    daily["date"] = daily["date"].dt.strftime("%Y-%m-%d")

    return daily


# Merge monthly NDVI from all years
ndvi_monthly = merge_ndvi_years(NDVI_FILES)
ndvi_monthly.to_csv(OUT_MONTHLY, index=False)
print("Saved merged monthly NDVI:", OUT_MONTHLY, ndvi_monthly.shape)

# Expand monthly NDVI → daily NDVI
ndvi_daily = monthly_to_daily_forwardfill(ndvi_monthly, START_DATE, END_DATE)
ndvi_daily.to_csv(OUT_DAILY, index=False)
print("Saved daily NDVI:", OUT_DAILY, ndvi_daily.shape)

# Sanity checks
print("Unique zones:", ndvi_monthly["zone_id"].nunique())
print("Monthly rows (expected 35*36=1260):", len(ndvi_monthly))
print("Daily rows (expected 35*1095=38325):", len(ndvi_daily))
print(ndvi_monthly.head())
print(ndvi_daily.head())


Saved merged monthly NDVI: /content/satellite_zone_monthly_ndvi_2021_2023.csv (1260, 4)
Saved daily NDVI: /content/satellite_zone_daily_2021_2023.csv (38325, 3)
Unique zones: 35
Monthly rows (expected 35*36=1260): 1260
Daily rows (expected 35*1095=38325): 38325
   zone_id       date      ndvi  year_source
0  bumbogo 2021-01-01  0.534608         2021
1  bumbogo 2021-02-01  0.307311         2021
2  bumbogo 2021-03-01  0.468052         2021
3  bumbogo 2021-04-01  0.489327         2021
4  bumbogo 2021-05-01  0.454134         2021
   zone_id        date      ndvi
0  bumbogo  2021-01-01  0.534608
1  bumbogo  2021-01-02  0.534608
2  bumbogo  2021-01-03  0.534608
3  bumbogo  2021-01-04  0.534608
4  bumbogo  2021-01-05  0.534608


**Merge all Integrated Dataset**

In [ ]:
import pandas as pd

# Load datasets
baseline = pd.read_csv("/content/drive/MyDrive/WI-DR/historical_waste_kigali_sectors.csv")
events   = pd.read_csv("/content/drive/MyDrive/WI-DR/events_calendar_zone_daily_2021_2023.csv")
weather  = pd.read_csv("/content/drive/MyDrive/WI-DR/weather_daily_kigali.csv")
ndvi     = pd.read_csv("/content/drive/MyDrive/WI-DR/satellite_zone_daily_2021_2023.csv")

def clean_text(s):
    return (s.astype(str)
             .str.strip()
             .str.lower()
             .str.replace(r"\s+", " ", regex=True))

# Standardize dates + ids
for df in [baseline, events, weather, ndvi]:
    df["date"] = pd.to_datetime(df["date"])
    if "zone_id" in df.columns:
        df["zone_id"] = df["zone_id"].astype(str).str.strip()

# Weather is city-level → replicate across all zones
zones = baseline[["zone_id"]].drop_duplicates()
weather_z = zones.merge(weather, how="cross")

# Merge baseline + events + weather first
integrated = (baseline
              .merge(events,    on=["zone_id","date"], how="left")
              .merge(weather_z, on=["zone_id","date"], how="left"))


# create a shared cleaned key and merge on (zone_key + date)
if "zone_name" not in integrated.columns:
    raise ValueError("baseline must contain a 'zone_name' column to merge NDVI correctly.")

integrated["zone_key"] = clean_text(integrated["zone_name"])
ndvi["zone_key"] = clean_text(ndvi["zone_id"])

# Merge NDVI using zone_key + date
integrated = integrated.merge(
    ndvi[["zone_key", "date", "ndvi"]],
    on=["zone_key", "date"],
    how="left"
)

# Drop helper column
integrated = integrated.drop(columns=["zone_key"])

# Save integrated dataset
OUT_FILE = "/content/drive/MyDrive/WI-DR/widr_zone_day_integrated_2021_2023.csv"
integrated.to_csv(OUT_FILE, index=False)

print("Integrated dataset saved:", OUT_FILE)
print("Shape:", integrated.shape)
print("NDVI missing %:", integrated["ndvi"].isna().mean() * 100)
print(integrated[["date","zone_id","zone_name","ndvi"]].head())


Integrated dataset saved: /content/drive/MyDrive/WI-DR/widr_zone_day_integrated_2021_2023.csv
Shape: (38325, 14)
NDVI missing %: 0.0
        date zone_id   zone_name      ndvi
0 2021-01-01  SEC_01      Gitega  0.245045
1 2021-01-01  SEC_02    Kanyinya  0.599476
2 2021-01-01  SEC_03      Kigali  0.494406
3 2021-01-01  SEC_04  Kimisagara  0.294643
4 2021-01-01  SEC_05  Mageregere  0.468142
